In [50]:
import xarray as xr
import numpy as np
import datetime as dt
import warnings
#Ignore UserWarning from xarray
warnings.filterwarnings("ignore", category=UserWarning)


In [51]:
flight = 'rf06'
output_filename = '/scr/raf_data/CAESAR/CAESAR'+flight+'_merge.nc'

In [52]:
ds = xr.open_dataset(f'/scr/raf_data/CAESAR/CAESAR{flight}.nc') ##Open the data file
ds2 =xr.open_dataset(f'/scr/raf_data/CAESAR/CAESAR{flight}_WVISO.nc') ##Open the WVISO data file

In [54]:
parsed_date = dt.datetime.strptime(ds.FlightDate, '%m/%d/%Y').strftime('%Y-%m-%dT00:00:00') ## Get the flight date to convert the time as seconds since midnight
##Convert the time to seconds since midnight
reference_date = np.datetime64(parsed_date)
time_deltas = ds2.Time.astype('timedelta64[s]')
datetime_values = reference_date + time_deltas
ds2['Time'] = datetime_values

'2024-03-12T00:00:00'

In [57]:
#Match up the time of the two datasets
ds2 = ds2.sel(Time =slice(ds.Time[0], ds.Time[-1]))

In [72]:
#Replace the WVISO variables in the original dataset with the ones from the WVISO dataset
for v in ds2.variables:
    if v == 'Time':
        continue
    ds[v].values = ds2[v].values ##Only replace the values -- keep attributes from original dataset
    if 'actual_range' in ds[v].attrs:
        minim = ds[v].min().values
        maxim = ds[v].max().values
        ds[v].attrs['actual_range'] =np.array([minim,maxim],dtype='float32') ##Update the actual range attribute

In [12]:
ds.to_netcdf(output_filename) ##Save the merged dataset